[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mongodb-developer/GenAI-Showcase/blob/main/notebooks/rag/rag_with_hugging_face_gemma_mongodb.ipynb)
[![View Article](https://img.shields.io/badge/View%20Article-blue)](https://www.mongodb.com/developer/products/atlas/gemma-mongodb-huggingface-rag/)



In [1]:
!pip install datasets pandas pymongo sentence_transformers
!pip install -U transformers
# Install below if using GPU
!pip install accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 55.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/base_command.py", line 179, in 
exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/req_command.py", line 67, in 
wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/commands/install.py", line 377, in run
    requirement_set = resolve

In [ ]:
# Load Dataset
import pandas as pd
from datasets import load_dataset

# https://huggingface.co/datasets/MongoDB/embedded_movies
dataset = load_dataset("MongoDB/embedded_movies")

# Convert the dataset to a pandas dataframe
dataset_df = pd.DataFrame(dataset["train"])

dataset_df.head(5)

,num_mflix_comments,genres,countries,directors,fullplot,writers,awards,runtime,type,rated,metacritic,poster,languages,imdb,plot,cast,plot_embedding,title
0,0,[Action],[USA],"[Louis J. Gasnier, Donald MacKenzie]",Young Pauline is left a lot of money when her ...,"[Charles W. Goddard (screenplay), Basil Dickey...","{'nominations': 0, 'text': '1 win.', 'wins': 1}",199.0,movie,None,NaN,https://m.media-amazon.com/images/M/MV5BMzgxOD...,[English],"{'id': 4465, 'rating': 7.6, 'votes': 744}",Young Pauline is left a lot of money when her ...,"[Pearl White, Crane Wilbur, Paul Panzer, Edwar...","[0.00072939653, -0.026834568, 0.013515796, -0....",The Perils of Pauline
1,0,"[Comedy, Short, Action]",[USA],"[Alfred J. Goulding, Hal Roach]",As a penniless man worries about how he will m...,[H.M. Walker (titles)],"{'nominations': 1, 'text': '1 nomination.', 'w...",22.0,movie,TV-G,NaN,https://m.media-amazon.com/images/M/MV5BNzE1OW...,[English],"{'id': 10146, 'rating': 7.0, 'votes': 639}",A penniless young man tries to save an heiress...,"[Harold Lloyd, Mildred Davis, 'Snub' Pollard, ...","[-0.022837115, -0.022941574, 0.014937485, -0.0...",From Hand to Mouth
2,0,"[Action, Adventure, Drama]",[USA],[Herbert Brenon],"Michael ""Beau"" Geste leaves England in disgrac...","[Herbert Brenon (adaptation), John Russell (ad...","{'nominations': 0, 'text': '1 win.', 'wins': 1}",101.0,movie,None,NaN,None,[English],"{'id': 16634, 'rating': 6.9, 'votes': 222}","Michael ""Beau"" Geste leaves England in disgrac...","[Ronald Colman, Neil Hamilton, Ralph Forbes, A...","[0.00023330493, -0.028511643, 0.014653289, -0....",Beau Geste
3,1,"[Adventure, Action]",[USA],[Albert Parker],A nobleman vows to avenge the death of his fat...,"[Douglas Fairbanks (story), Jack Cunningham (a...","{'nominations': 0, 'text': '1 win.', 'wins': 1}",88.0,movie,None,NaN,https://m.media-amazon.com/images/M/MV5BMzU0ND...,None,"{'id': 16654, 'rating': 7.2, 'votes': 1146}","Seeking revenge, an athletic young man joins t...","[Billie Dove, Tempe Pigott, Donald Crisp, Sam ...","[-0.005927917, -0.033394486, 0.0015323418, -0....",The Black Pirate
4,0,"[Action, Comedy, Romance]",[USA],[Sam Taylor],"The Uptown Boy, J. Harold Manners (Lloyd) is a...","[Ted Wilde (story), John Grey (story), Clyde B...","{'nominations': 1, 'text': '1 nomination.', 'w...",58.0,movie,PASSED,NaN,https://m.media-amazon.com/images/M/MV5BMTcxMT...,[English],"{'id': 16895, 'rating': 7.6, 'votes': 918}",An irresponsible young millionaire changes his...,"[Harold Lloyd, Jobyna Ralston, Noah Young, Jim...","[-0.0059373598, -0.026604708, -0.0070914757, -...",For Heaven's Sake


In [ ]:
# Data Preparation

# Remove data point where plot coloumn is missing
dataset_df = dataset_df.dropna(subset=["fullplot"])
print("\nNumber of missing values in each column after removal:")
print(dataset_df.isnull().sum())

# Remove the plot_embedding from each data point in the dataset as we are going to create new embeddings with an open source embedding model from Hugging Face
dataset_df = dataset_df.drop(columns=["plot_embedding"])
dataset_df.head(5)


Number of missing values in each column after removal:
num_mflix_comments      0
genres                  0
countries               0
directors              12
fullplot                0
writers                13
awards                  0
runtime                14
type                    0
rated                 279
metacritic            893
poster                 78
languages               1
imdb                    0
plot                    0
cast                    1
plot_embedding          1
title                   0
dtype: int64


,num_mflix_comments,genres,countries,directors,fullplot,writers,awards,runtime,type,rated,metacritic,poster,languages,imdb,plot,cast,title
0,0,[Action],[USA],"[Louis J. Gasnier, Donald MacKenzie]",Young Pauline is left a lot of money when her ...,"[Charles W. Goddard (screenplay), Basil Dickey...","{'nominations': 0, 'text': '1 win.', 'wins': 1}",199.0,movie,None,NaN,https://m.media-amazon.com/images/M/MV5BMzgxOD...,[English],"{'id': 4465, 'rating': 7.6, 'votes': 744}",Young Pauline is left a lot of money when her ...,"[Pearl White, Crane Wilbur, Paul Panzer, Edwar...",The Perils of Pauline
1,0,"[Comedy, Short, Action]",[USA],"[Alfred J. Goulding, Hal Roach]",As a penniless man worries about how he will m...,[H.M. Walker (titles)],"{'nominations': 1, 'text': '1 nomination.', 'w...",22.0,movie,TV-G,NaN,https://m.media-amazon.com/images/M/MV5BNzE1OW...,[English],"{'id': 10146, 'rating': 7.0, 'votes': 639}",A penniless young man tries to save an heiress...,"[Harold Lloyd, Mildred Davis, 'Snub' Pollard, ...",From Hand to Mouth
2,0,"[Action, Adventure, Drama]",[USA],[Herbert Brenon],"Michael ""Beau"" Geste leaves England in disgrac...","[Herbert Brenon (adaptation), John Russell (ad...","{'nominations': 0, 'text': '1 win.', 'wins': 1}",101.0,movie,None,NaN,None,[English],"{'id': 16634, 'rating': 6.9, 'votes': 222}","Michael ""Beau"" Geste leaves England in disgrac...","[Ronald Colman, Neil Hamilton, Ralph Forbes, A...",Beau Geste
3,1,"[Adventure, Action]",[USA],[Albert Parker],A nobleman vows to avenge the death of his fat...,"[Douglas Fairbanks (story), Jack Cunningham (a...","{'nominations': 0, 'text': '1 win.', 'wins': 1}",88.0,movie,None,NaN,https://m.media-amazon.com/images/M/MV5BMzU0ND...,None,"{'id': 16654, 'rating': 7.2, 'votes': 1146}","Seeking revenge, an athletic young man joins t...","[Billie Dove, Tempe Pigott, Donald Crisp, Sam ...",The Black Pirate
4,0,"[Action, Comedy, Romance]",[USA],[Sam Taylor],"The Uptown Boy, J. Harold Manners (Lloyd) is a...","[Ted Wilde (story), John Grey (story), Clyde B...","{'nominations': 1, 'text': '1 nomination.', 'w...",58.0,movie,PASSED,NaN,https://m.media-amazon.com/images/M/MV5BMTcxMT...,[English],"{'id': 16895, 'rating': 7.6, 'votes': 918}",An irresponsible young millionaire changes his...,"[Harold Lloyd, Jobyna Ralston, Noah Young, Jim...",For Heaven's Sake


In [ ]:
from sentence_transformers import SentenceTransformer

# https://huggingface.co/thenlper/gte-large
embedding_model = SentenceTransformer("thenlper/gte-large")


def get_embedding(text: str) -> list[float]:
    if not text.strip():
        print("Attempted to get embedding for empty text.")
        return []

    embedding = embedding_model.encode(text)

    return embedding.tolist()


dataset_df["embedding"] = dataset_df["fullplot"].apply(get_embedding)

dataset_df.head()

,num_mflix_comments,genres,countries,directors,fullplot,writers,awards,runtime,type,rated,metacritic,poster,languages,imdb,plot,cast,title,embedding
0,0,[Action],[USA],"[Louis J. Gasnier, Donald MacKenzie]",Young Pauline is left a lot of money when her ...,"[Charles W. Goddard (screenplay), Basil Dickey...","{'nominations': 0, 'text': '1 win.', 'wins': 1}",199.0,movie,None,NaN,https://m.media-amazon.com/images/M/MV5BMzgxOD...,[English],"{'id': 4465, 'rating': 7.6, 'votes': 744}",Young Pauline is left a lot of money when her ...,"[Pearl White, Crane Wilbur, Paul Panzer, Edwar...",The Perils of Pauline,"[-0.009285838343203068, -0.005062104668468237,..."
1,0,"[Comedy, Short, Action]",[USA],"[Alfred J. Goulding, Hal Roach]",As a penniless man worries about how he will m...,[H.M. Walker (titles)],"{'nominations': 1, 'text': '1 nomination.', 'w...",22.0,movie,TV-G,NaN,https://m.media-amazon.com/images/M/MV5BNzE1OW...,[English],"{'id': 10146, 'rating': 7.0, 'votes': 639}",A penniless young man tries to save an heiress...,"[Harold Lloyd, Mildred Davis, 'Snub' Pollard, ...",From Hand to Mouth,"[-0.0024393785279244184, 0.02309592440724373, ..."
2,0,"[Action, Adventure, Drama]",[USA],[Herbert Brenon],"Michael ""Beau"" Geste leaves England in disgrac...","[Herbert Brenon (adaptation), John Russell (ad...","{'nominations': 0, 'text': '1 win.', 'wins': 1}",101.0,movie,None,NaN,None,[English],"{'id': 16634, 'rating': 6.9, 'votes': 222}","Michael ""Beau"" Geste leaves England in disgrac...","[Ronald Colman, Neil Hamilton, Ralph Forbes, A...",Beau Geste,"[0.012204292230308056, -0.01145575474947691, -..."
3,1,"[Adventure, Action]",[USA],[Albert Parker],A nobleman vows to avenge the death of his fat...,"[Douglas Fairbanks (story), Jack Cunningham (a...","{'nominations': 0, 'text': '1 win.', 'wins': 1}",88.0,movie,None,NaN,https://m.media-amazon.com/images/M/MV5BMzU0ND...,None,"{'id': 16654, 'rating': 7.2, 'votes': 1146}","Seeking revenge, an athletic young man joins t...","[Billie Dove, Tempe Pigott, Donald Crisp, Sam ...",The Black Pirate,"[0.004541348200291395, -0.0006100579630583525,..."
4,0,"[Action, Comedy, Romance]",[USA],[Sam Taylor],"The Uptown Boy, J. Harold Manners (Lloyd) is a...","[Ted Wilde (story), John Grey (story), Clyde B...","{'nominations': 1, 'text': '1 nomination.', 'w...",58.0,movie,PASSED,NaN,https://m.media-amazon.com/images/M/MV5BMTcxMT...,[English],"{'id': 16895, 'rating': 7.6, 'votes': 918}",An irresponsible young millionaire changes his...,"[Harold Lloyd, Jobyna Ralston, Noah Young, Jim...",For Heaven's Sake,"[-0.0022256041411310434, 0.011567804962396622,..."


In [ ]:
import pymongo
from google.colab import userdata


def get_mongo_client(mongo_uri):
    """Establish connection to the MongoDB."""
    try:
        client = pymongo.MongoClient(
            mongo_uri, appname="devrel.showcase.rag_huggingface_gemma"
        )
        print("Connection to MongoDB successful")
        return client
    except pymongo.errors.ConnectionFailure as e:
        print(f"Connection failed: {e}")
        return None


mongo_uri = userdata.get("MONGO_URI")
if not mongo_uri:
    print("MONGO_URI not set in environment variables")

mongo_client = get_mongo_client(mongo_uri)

# Ingest data into MongoDB
db = mongo_client["movies"]
collection = db["movie_collection_2"]

Connection to MongoDB successful


In [ ]:
# Delete any existing records in the collection
collection.delete_many({})

DeleteResult({'n': 1452, 'electionId': ObjectId('7fffffff000000000000000c'), 'opTime': {'ts': Timestamp(1708554945, 1452), 't': 12}, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1708554945, 1452), 'signature': {'hash': b'\x99\x89\xc0\x00Cn!\xd6\xaf\xb3\x96\xdf\xc3\xda\x88\x11\xf5\t\xbd\xc0', 'keyId': 7320226449804230661}}, 'operationTime': Timestamp(1708554945, 1452)}, acknowledged=True)

In [ ]:
documents = dataset_df.to_dict("records")
collection.insert_many(documents)

print("Data ingestion into MongoDB completed")

Data ingestion into MongoDB completed


In [ ]:
def vector_search(user_query, collection):
    """
    Perform a vector search in the MongoDB collection based on the user query.

    Args:
    user_query (str): The user's query string.
    collection (MongoCollection): The MongoDB collection to search.

    Returns:
    list: A list of matching documents.
    """

    # Generate embedding for the user query
    query_embedding = get_embedding(user_query)

    if query_embedding is None:
        return "Invalid query or embedding generation failed."

    # Define the vector search pipeline
    vector_search_stage = {
        "$vectorSearch": {
            "index": "vector_index",
            "queryVector": query_embedding,
            "path": "embedding",
            "numCandidates": 150,  # Number of candidate matches to consider
            "limit": 4,  # Return top 4 matches
        }
    }

    unset_stage = {
        "$unset": "embedding"  # Exclude the 'embedding' field from the results
    }

    project_stage = {
        "$project": {
            "_id": 0,  # Exclude the _id field
            "fullplot": 1,  # Include the plot field
            "title": 1,  # Include the title field
            "genres": 1,  # Include the genres field
            "score": {"$meta": "vectorSearchScore"},  # Include the search score
        }
    }

    pipeline = [vector_search_stage, unset_stage, project_stage]

    # Execute the search
    results = collection.aggregate(pipeline)
    return list(results)

In [ ]:
def get_search_result(query, collection):
    get_knowledge = vector_search(query, collection)

    search_result = ""
    for result in get_knowledge:
        search_result += f"Title: {result.get('title', 'N/A')}, Plot: {result.get('fullplot', 'N/A')}\n"

    return search_result

In [ ]:
# Conduct query with retrival of sources
query = "What is the best romantic movie to watch and why?"
source_information = get_search_result(query, collection)
combined_information = f"Query: {query}\nContinue to answer the query by using the Search Results:\n{source_information}."

print(combined_information)

Query: What is the best romantic movie to watch and why?
Continue to answer the query by using the Search Results:
Title: Shut Up and Kiss Me!, Plot: Ryan and Pete are 27-year old best friends in Miami, born on the same day and each searching for the perfect woman. Ryan is a rookie stockbroker living with his psychic Mom. Pete is a slick surfer dude yet to find commitment. Each meets the women of their dreams on the same day. Ryan knocks heads in an elevator with the gorgeous Jessica, passing out before getting her number. Pete falls for the insatiable Tiara, but Tiara's uncle is mob boss Vincent Bublione, charged with her protection. This high-energy romantic comedy asks to what extent will you go for true love?
Title: Pearl Harbor, Plot: Pearl Harbor is a classic tale of romance set during a war that complicates everything. It all starts when childhood friends Rafe and Danny become Army Air Corps pilots and meet Evelyn, a Navy nurse. Rafe falls head over heels and next thing you know

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it")
# CPU Enabled uncomment below 👇🏽
# model = AutoModelForCausalLM.from_pretrained("google/gemma-2b-it")
# GPU Enabled use below 👇🏽
model = AutoModelForCausalLM.from_pretrained("google/gemma-2b-it", device_map="auto")

In [ ]:
# Moving tensors to GPU
input_ids = tokenizer(combined_information, return_tensors="pt").to("cuda")
response = model.generate(**input_ids, max_new_tokens=500)
print(tokenizer.decode(response[0]))

<bos>Query: What is the best romantic movie to watch and why?
Continue to answer the query by using the Search Results:
Title: Shut Up and Kiss Me!, Plot: Ryan and Pete are 27-year old best friends in Miami, born on the same day and each searching for the perfect woman. Ryan is a rookie stockbroker living with his psychic Mom. Pete is a slick surfer dude yet to find commitment. Each meets the women of their dreams on the same day. Ryan knocks heads in an elevator with the gorgeous Jessica, passing out before getting her number. Pete falls for the insatiable Tiara, but Tiara's uncle is mob boss Vincent Bublione, charged with her protection. This high-energy romantic comedy asks to what extent will you go for true love?
Title: Pearl Harbor, Plot: Pearl Harbor is a classic tale of romance set during a war that complicates everything. It all starts when childhood friends Rafe and Danny become Army Air Corps pilots and meet Evelyn, a Navy nurse. Rafe falls head over heels and next thing you